# Libs

In [1]:
import numpy as np
from os.path import join
import random
from torchvision import transforms

In [ ]:
GDRIVE_PATHS = {
    'main': '/content/gdrive/MyDrive/trashbean-classifier/',
    'dataset': '/content/gdrive/MyDrive/trashbean-classifier/dataset/',
    'logs': '/content/gdrive/MyDrive/trashbean-classifier/logs/',
    'models': '/content/gdrive/MyDrive/trashbean-classifier/logs/models/',
    'libs': '/content/gdrive/MyDrive/Colab Notebooks/Progetto/libs'
}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Insert the directory
import sys
sys.path.insert(0,GDRIVE_PATHS['libs'])

In [6]:
# need do import only concreteCreator
from TrashbeanDataset import TrashbeanDataset
from TDContainer import TDContainer
from PretrainedModels import AverageValueMeter, PretrainedModelsCreator, CCAlexNet, CCSqueezeNet, CCVgg16

In [3]:
random.seed(1996)
np.random.seed(1996)

# Testing

## Load del dataset

In [7]:
train_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.RandomCrop(224),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224), # crop centrale
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dst_train = {
  'path': join(GDRIVE_PATHS['dataset'], 'training.csv'),
  'transform': train_transform
}

dst_validation = {
  'path': join(GDRIVE_PATHS['dataset'], 'validation.csv'),
  'transform': test_transform
}

dst_test = {
  'path': join(GDRIVE_PATHS['dataset'], 'test.csv'),
  'transform': test_transform
}

trashbean_dataset = TDContainer(training=dst_train, validation=dst_validation, test=dst_test)

In [ ]:
trashbean_dataset.show_info()

## Training

In [9]:
def do_training(creator: PretrainedModelsCreator, model_name: str="default_model", dataset: TDContainer, output_class: int=3) -> None:
    creator.initialize_dst(dataset, output_class, batch_size=32, num_workers=2, drop_last=False)
    creator.train(model_name=model_name, lr=0.01, epochs=2, momemtum=0.99, log_dir=GDRIVE_PATHS['logs'], model_dir=GDRIVE_PATHS['models'], train_from_epoch=0, save_on_runtime=False, save_each_iter=1)

In [ ]:
# using the Concrete Creator
print("App: Launching training with the SqueezeNet.")
do_training(creator=CCSqueezeNet(), model_name="SQUEEZENET_v0.1", dataset=trashbean_dataset, output_class=3)

In [ ]:
print("App: Launching training with the AlexNet")
do_training(creator=CCAlexNet(), model_name="ALEXNET_v0.1", dataset=trashbean_dataset, output_class=3)

In [ ]:
print("App: Launching training with the VGG16")
do_training(creator=CCVgg16(), model_name="VGG16_v0.1", dataset=trashbean_dataset, output_class=3)

In [ ]:
### do loading....

In [15]:
def make_model_name(path_model, name_model, version):
  return join(path_model, "%s__%s.pth" % (name_model, version))

name_model = 'SqueezeNet'
version="1-0"
print(make_model_path(GDRIVE_PATHS['models'], "SqueezeNet", "v0-999"))